# Crypto Lab 2: Putting it all together

**Prerequisites:**
- Complete all parts 1-5 of this lab
- Complemte the IoT lab

Now the time has come to put everything together. Your task is to create your own secret messaging scheme for the Micro:Bit.

- Part 1: Verify that the radio works. I.e., two microbits can communicate.
- Part 2: Using ChaCha20, we begin securing the communication between two Micro:Bits. We use a static key to begin with and provide code for the receiver-side. 
- Part 3: Extend with MAC functionality. We provide a custom hashing algortihm for the MAC.  
- Part 4: Extend so that the key is no longer static, but computed using DHKE. 



---

> **NB: Make sure you have completed Crypto Lab 1: Basic introductions before starting on this lab**

In this lab, you’ll take what you learned in the previous crypto lab and bring it to life on real hardware using the **Micro:Bit**. You’ll also be introduced to a new algorithm: **ChaCha20** that you've seen in the preparation materials. 

To follow along, you’ll need two Micro:Bits - ask your teacher assistants's if you only have one. 

## 1 Verify that two Micro:Bits can communicate using the Radio

First, let’s make sure the two Micro:Bits can communicate with each other!  
Copy the code provided in [lab2-1.py](/crypto-lab/lab-2/lab2-1.py) into the [Micro:Bit Web Editor](https://python.microbit.org/v/3) and upload it to **both** of your devices.  

Next, choose one Micro:Bit to act as the **sender** and the other as the **receiver**.  
- On the sender, press the buttons to transmit a message.  
- On the receiver, check that the message is displayed correctly.  

> ⚠️ **Important:** Before you run the program, update the `GROUP_NUMBER` in the code to match your assigned rat group.  
> Your group number is created by combining your team number with A/B (where A = 0 and B = 1).  

Examples:  
- Team **1A** → `10`  
- Team **1B** → `11` 

#### Question 1.1: Explain briefly what happens in the code snippet (maximum 3 lines of text)

#### Question 1.2: Right now, the messages are sent “in the clear” over the radio - what does this mean?

#### Question 1.3: What happens if another team nearby sets the same group number?

Before moving on, let’s make sure the Micro:Bits can talk to each other:  

- Choose one Micro:Bit as the **sender** and the other as the **receiver**.  
- Press **A** or **B** on the sender.  
- Check that the corresponding message appears on the receiver’s display.  

✅ Verify that the basic communication works correctly before proceeding to encryption.

---

## 2 Encryption and decryption of messages

Now that you’ve confirmed the two Micro:Bits can communicate, it’s time to take the next step: implementing a simple **encryption and decryption** mechanism.  

1. Copy the code from [lab2-2.py](/crypto-lab/lab-2/lab2-2.py) into the [Micro:Bit Web Editor](https://python.microbit.org/v/3).  
2. ⚠️ Don’t forget to update the `GROUP_NUMBER` here as well, just like in Task 1.  
3. Send the code to both Micro:Bits. 

If you look at the new code, you’ll notice that most of the previous functions remain unchanged.  
However, a few **new elements have been added**!

If you scroll down to the bottom of the Python file, you’ll find a **ChaCha20 module**.  
Don’t worry if it looks complex, you are **not expected to fully understand it** in this course - however it will become relevant in later cryptographic courses.

As seen in the preparation material, ChaCha20 is a widely used stream cipher.

#### Question 2.1: Why does ChaCha20 only have an encrypt function, not decrypt? 

You’ll also notice some **new constants** in the code:  

- `GLOBAL_KEY` → a static global key written in binary format  
- `MESSAGE_1` / `MESSAGE_2` → example messages you can use to test the functionality  

💡 **Why this matters:** In real-world cryptography, securely sharing and managing keys is one of the hardest challenges.  

For now, we use a fixed key for simplicity, but later you’ll see how key exchange and management become critical for secure communication.  

In [9]:
GLOBAL_KEY = b'chacha20!'
MESSAGE_1 = "Hello World"
MESSAGE_2 = "Goodbye World"

Furthermore, both functions:`def send_mode()` and `def receive_mode()` are altered. They both call a new function:`on_receive()` before displaying the data: 

In [ ]:
def on_receive(received_bytes):
    try: 
        decrypted_bytes = chacha20_encrypt(
            received_bytes, GLOBAL_KEY)
        data = decrypted_bytes.decode('utf-8')
        return data
    except TypeError:
        return "TYPE ERROR"
    except:
        return "WEIRD ERROR"

#### Question 2.2: Explain shortly what happens in the `on_receive` function.

Additionally, `def send_mode()` has been altered to send the messages written as constants instead of "A" or "B" as in part 1. Here, a new function `on_send()` is used instead of `radio.send()`.

Using the `on_receive()` function, let's try to fill out `on_send()`!   

#### Task: Follow the steps as written in the TODO and copy your solution into the code block below 

In [ ]:
def on_send(msg):
    """
    TODO: 
    Implement a send function that encodes the inputted
    message to bytes, encrypts the bytes using chacha20_encrypt()
    and then sends the encrypted bytes over the radio.
    """
    pass # Remove this line and implement your solution here

<details>
<summary><strong>💡 Hint</strong></summary>

Try using the same logic as in on_receive() to fill out on_send(). NOEN BEDRE HINT HER???
</details>

Now that you’ve added encryption, it’s time to test your code:  

- Send a message from one Micro:Bit.  
- On the receiving side, the message should be **decrypted and displayed correctly**.  
- Make sure the communication works for both `MESSAGE_1` and `MESSAGE_2`.  

✅ Verify that your devices can still communicate and that your encrypted messages are being properly sent, received, and decrypted.

## 3 Extend with MAC technology



Great job! Now that the simple encryption through ChaCha20 is in place, let's further implement MAC (Message Authentication Code):

1. Copy the code from [lab2-3.py](/crypto-lab/lab-2/lab2-3.py) into the [Micro:Bit Web Editor](https://python.microbit.org/v/3).  
2. ⚠️ Don’t forget to update the `GROUP_NUMBER` here as well, just like in Task 1 and 2. 
3. ⚠️ Make sure to copy the code from task 2 into the function `on_send()`.
4. Send the code to both Micro:Bits.  

Similarily as in the last task, some parts of the code is now altered!

Firstly, let's look at the changes made in `on_receive()`: 

In [ ]:
def on_receive(received_bytes):
    try:
        mac, msg = split_data(received_bytes)
        decrypted_bytes = chacha20_encrypt(
            msg, GLOBAL_KEY)
        data = decrypted_bytes.decode('utf-8')

        if verify_mac(GLOBAL_KEY.decode('utf-8'),
                  data,
                  mac):
            return data
        else:
            return "INVALID MAC"
    except TypeError:
        return "TYPE ERROR"
    except:
        return "UNEXPECTED ERROR"
    
def split_data(data):
    mac = data[0:2]
    msg = data[2:]
    return mac, msg

The `on_receive()` function has been updated to include **verification of the MAC** sent over the radio.  

Here’s what happens step by step:  
1. The incoming data is split into two parts using the `split_data()` function:  
   - the fixed-length **MAC**  
   - the variable-length **message**  
2. The message is then **decrypted** and converted back into a readable string.  
3. Finally, the `verify_mac()` function checks whether the MAC matches the message.  
   - If it does, the message should be returned.  
   - If not, an error is shown (e.g., `"INVALID MAC"`).  

However, this `verify_mac()` function is not yet created. Neither is `generate_mac()`! A simple hash function is created to help you implement these functions. Try to understand how the function works, but don't spend too much time on details. 

#### Question 3.1: In this lab, we created `simple_hash()` ourselves. Why is writing your own cryptographic functions usually a bad idea?

In [1]:
# Fill in your answer here.

Unfortunately, due to environmental limitations we will have to use the simple_hash() in this case.

Using the provided `simple_hash()` function, implement message authentication code functionality:

- **`generate_mac(key, message)`** — return a MAC (bytes) for the given key and message.  
- **`verify_mac(key, message, mac)`** — return `True` if `mac` is valid for the given key and message, otherwise `False`.

Your MAC should be **prepended** to the encrypted bytes in `on_send()` and **checked** in `on_receive()` before accepting a message.

#### Task: Create the functionality for `generate_mac()` and copy your solution into the code block below:



In [ ]:
def generate_mac(key, message):
    """
    TODO:
    Implement a function to create a message authentication code (MAC) using the provided key and message.
    """

<details>
<summary><strong>💡 Hint for generate_mac()</strong></summary>

1. Think about how you can **combine the key and the message** into a single string.  
2. Pass that combined string into the `simple_hash()` function.  
3. Return the resulting bytes as the MAC. 
</details>

#### Task: Create the functionality for `verify_mac()` and copy your solution into the code block below:

In [ ]:
def verify_mac(key, message, mac):
    """
    TODO:
    Implement a function to verify a message authentication code (MAC) using the provided key, message, and MAC.
    This function should return True if the MAC is valid, and False otherwise.
    """

<details>
<summary><strong>💡 Hint for verify_mac()</strong></summary>

1. Recreate the MAC yourself by calling `generate_mac(key, message)`.  
2. Compare the recreated MAC with the one received.  
3. Return `True` if they match, otherwise `False`.  
</details>

Now it’s time to try sending messages again!  
Upload your updated code to the Micro:Bits and test it:  

- When you send a message, a **MAC should be generated and attached**.  
- On the receiving side, the **MAC should be verified** before the message is shown.  

✅ Verify that your devices can still communicate and that the MAC verification works correctly.  


## 4 Diffie-Hellman Key Exchange (DHKE)

In the last part of this lab, you will extend your program to use **Diffie–Hellman Key Exchange** instead of the static key you’ve been relying on so far.  

If you need a refresher on how Diffie–Hellman works, take a look back at **Cryptography Lab 1** before starting this task.  


1. Copy the code from [lab2-4.py](/crypto-lab/lab-2/lab2-4.py) into the [Micro:Bit Web Editor](https://python.microbit.org/v/3).  
2. ⚠️ Don’t forget to update the `GROUP_NUMBER` here as well, just like previous tasks. 
3. ⚠️ Make sure to copy the code from task 2 into the function `on_send()`.
4. ⚠️ Include the code you filled in from task 3 for the MAC-calculation. 
4. Send the code to both Micro:Bits.  

Let's have a look at what parts is altered:

First, and importantly, the key is set to None

In [2]:
GLOBAL_KEY = None

Further down (in the DHKE section), constants for DHKE is included as well as a TODO for the last part of the lab. Read this carefully before starting! 

In [ ]:
# Constants for DHKE
g = 5
p = 97

Below, you find the new functions where you have to fill in some functionalities. 

#### Task: Create the functionality to complete `initialize_send_mode`, `initialize_receive_mode`, and `generate_A`, `generate_B` and `generate_K`:

In [ ]:
def initialize_send_mode():
    """
    TASK: Fix DHKE sender protocol
    TODO:
    1. Generate random private key 'a' (integer from 1 to p-1)
    2. Calculate public key A = g^a mod p using generate_A()
    3. Send A to receiver as string
    4. Wait for B from receiver
    5. Calculate shared secret using generate_and_set_K()
    6. Set GLOBAL_KEY and start send_mode()
    """
    global GLOBAL_KEY
    
    # TODO: Fix random number generation 
    a = 0 # generate the private share "a" random integer from 1 to p-1
    A = 0 # generate the public share A using the private a, g and p

    display.scroll("DHKE SEND")
    display.scroll("A=" + str(A))
    
    attempts = 0
    max_attempts = 20
    
    while GLOBAL_KEY is None and attempts < max_attempts:
        # Send A as string via radio using UTF-8 encoding
        radio.send_bytes(str(A).encode('utf-8')) 
        
        sleep(5000)  # Wait 5 seconds

        # Wait for B from receiver
        received_msg = radio.receive()
        if received_msg:
            try:
                B = int(received_msg)  
                
                # TODO: Calculate shared secret K
                K = 0 # Something
                GLOBAL_KEY = K
                
                display.scroll("K=" + str(K))
                display.show(Image.YES)
                break
            except:
                # Invalid message, continue trying
                pass
        
        attempts += 1

    if GLOBAL_KEY is not None:
        display.scroll("KEY OK!")
        send_mode()
    else:
        display.scroll("KEY FAIL!")

In [ ]:
def initialize_receive_mode():  
    """
    TASK: Fix DHKE receiver protocol  
    TODO:
    1. Generate random private key 'b' (integer from 1 to p-1)
    2. Wait for A from sender
    3. Calculate public key B = g^b mod p using generate_B()
    4. Send B to sender as string
    5. Calculate shared secret using generate_and_set_K()
    6. Set GLOBAL_KEY and start receive_mode()
    """
    global GLOBAL_KEY
    
    # TODO: Fix random number generation
    b = 0 # generate the private share "b" random integer from 1 to p-1
    
    display.scroll("DHKE RECV")
    
    attempts = 0
    max_attempts = 30
    
    while GLOBAL_KEY is None and attempts < max_attempts:
        # Wait for A from sender
        received_msg = radio.receive()
        if received_msg:
            try:
                # Parse A as integer
                A = int(received_msg)
                
                display.scroll("A=" + str(A))
                
                # TODO: Generate public key B
                B = 0 # generate the public share B using the private b, g and p
                display.scroll("B=" + str(B))
                
                # Send B to sender as string
                radio.send(str(B)) 
                
                # TODO: Calculate shared secret K
                K = 0 # generate the public share K using A, b and p
                GLOBAL_KEY = K
                
                display.scroll("K=" + str(K))
                display.show(Image.YES)
                break
            except:
                # Invalid message, continue waiting
                pass
        
        sleep(500)  # Check every 0.5 seconds
        attempts += 1
    
    if GLOBAL_KEY is not None:
        display.scroll("KEY OK!")
        receive_mode()
    else:
        display.scroll("KEY FAIL!")

In [ ]:
def generate_A(a, g, p):
    # TODO: Generate A
    A = 0 # A = g^a mod p
    return A

def generate_B(b, g, p):
    # TODO: Generate B
    B = 0 # B = g^b mod p
    return B

def generate_K(X, y, p):
    # TODO: Generate X
    K = 0 # K = X^y mod p
    return K

Endringer:
- Key = None
- Send-mode prøver å lage nøkkel og venter på svar før den sender
- Receive-mode venter på en initiell melding før den generer nøkkel og svarer
- begge har delt nøkkel

